In [47]:
import pickle as pkl
import numpy as np
import urllib.request
import cv2
import pandas as pd
from tqdm import tqdm

In [49]:
from sklearn.model_selection import train_test_split

In [2]:
model_clf_1 = pkl.load(open('models/model_1.pkl','rb'))
model_clf_2 = pkl.load(open('models/model_2.pkl','rb'))
model_clf_3 = pkl.load(open('models/model_3.pkl','rb'))
model_boost_1 = pkl.load(open('models/model-boost_1.pkl','rb'))
model_boost_2 = pkl.load(open('models/model-boost_2.pkl','rb'))

pipline1 = pkl.load(open('models/pipline_1.pkl','rb'))
labels1 = pkl.load(open('models/labels_1.pkl', 'rb'))
pipline2 = pkl.load(open('models/pipline_2.pkl','rb'))
labels2 = pkl.load(open('models/labels_2.pkl', 'rb'))
pipline3 = pkl.load(open('models/pipline_3.pkl','rb'))
labels3 = pkl.load(open('models/labels_3.pkl', 'rb'))

In [3]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
def adjusted_detect_face(img):
    face_img = img.copy()
    face_rect = face_cascade.detectMultiScale(face_img, scaleFactor=1.2, minNeighbors=5)

    if len(face_rect) == 0:
        return None

    x, y, w, h = face_rect[0]
    cropped_image = face_img[y:y+h, x:x+w]
    return cropped_image

In [4]:
def url_to_image(url):
    # Download the image, convert it to a NumPy array, and then decode it
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    # Use cv2.imdecode() to convert the array to an OpenCV image format
    image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE) 
    return image

In [5]:
image_url = "https://img-cdn.inc.com/image/upload/f_webp,c_fit,w_1920,q_auto/images/panoramic/getty_505175324_2000131020009280327_186071.jpg" # Replace with your image URL
image_from_url = url_to_image(image_url)

In [7]:
cropped_face = adjusted_detect_face(image_from_url)

In [8]:
# gray = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(cropped_face, (48, 48))
processed = resized.reshape(1, -1)

In [10]:
X1 = pipline1.transform(processed)
emotion = model_clf_1.predict(X1)
labels1.inverse_transform(emotion)

array(['Angry'], dtype=object)

In [11]:
X2 = pipline2.transform(processed)
emotion = model_clf_2.predict(X2)
labels2.inverse_transform(emotion)

array(['Happy'], dtype=object)

In [12]:
X3 = pipline3.transform(processed)
emotion = model_clf_3.predict(X3)
labels3.inverse_transform(emotion)

array(['Happy'], dtype=object)

In [13]:
emotion = model_boost_1.predict(X1)
labels1.inverse_transform(emotion)

array(['Happy'], dtype=object)

In [14]:
emotion = model_boost_2.predict(X2)
labels2.inverse_transform(emotion)

array(['Happy'], dtype=object)

In [15]:
models = {
    model_clf_1: [pipline1,labels1],
    model_clf_2: [pipline2,labels2],
    model_clf_3: [pipline3,labels3],
    model_boost_1: [pipline1,labels1],
    model_boost_2: [pipline2, labels2]
}

In [30]:
def combinepredic(models:dict,img):
    pred = []
    for model, pipline in models.items():
        X = pipline[0].transform(img)
        emotion = model.predict(X)
        pred.append(pipline[1].inverse_transform(emotion)[0])
    return pd.Series(pred).mode()[0]

In [31]:
pred = combinepredic(models, processed)

In [33]:
key = [f"{i}_px" for i in range(48*48)]
value = ['float32']*(48*48)
df = pd.read_csv('data.csv',dtype=dict(zip(key,value)))
del key, value
df.drop_duplicates(inplace=True)

In [34]:
X = df.iloc[:,:-1].to_numpy()
y = df['emotion'].to_numpy()

In [60]:
X.shape, y.shape

((28669, 2304), (28669,))

In [65]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)

In [66]:
X_test.shape

(5734, 2304)

In [67]:
pred = np.zeros((X_test.shape[0],)).astype(np.object_)
i = 0
for x in tqdm(X_test):
    pred[i] = combinepredic(models, x.reshape(1, -1))
    i+=1

100%|██████████| 5734/5734 [36:28<00:00,  2.62it/s]


In [68]:
from sklearn.preprocessing import LabelEncoder

In [69]:
lab_encoding = LabelEncoder()
y_ = lab_encoding.fit_transform(y_test)
pred_ = lab_encoding.transform(pred)

In [70]:
y_.shape, pred_.shape

((5734,), (5734,))

In [71]:
from sklearn.metrics import classification_report

In [72]:
print(classification_report(y_,pred_))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96       961
           1       0.99      0.92      0.95       969
           2       0.98      0.98      0.98      1883
           3       0.90      0.98      0.94      1245
           4       0.99      0.98      0.98       676

    accuracy                           0.96      5734
   macro avg       0.97      0.96      0.96      5734
weighted avg       0.96      0.96      0.96      5734

